# Lesson05. Body language - 讀懂機器人的內心

教學主題： 從機器人身上讀取數據。

核心目標：知道機器人也能「感知」自己的狀態。實現一個簡單的「輸入->處理->輸出」邏輯。

前 30 分鐘 (新視角)：
介紹 IMU（慣性測量單元），它是機器人的「小腦」，負責感知姿態。
學習讀取機器人身體的俯仰（pitch）和側傾（roll）角度。

後 90 分鐘 (互動實作)：數據監控： 
編寫程式，在電腦螢幕上即時顯示 Go2 的姿態數據。
【挑戰】 如果偵測到機器人被推倒（側傾角度大於 45 度），就觸發胸前 LED 警報。

## 5.1 Import and initialize the client

In [ ]:
import time
import sys
from unitree_sdk2py.core.channel import ChannelSubscriber, ChannelFactoryInitialize
from unitree_sdk2py.go2.sport.sport_client import SportClient
from unitree_sdk2py.idl.unitree_go.msg.dds_ import SportModeState_

ChannelFactoryInitialize(0, "ens37")
# ChannelFactoryInitialize(0)
sport_client = SportClient()
sport_client.SetTimeout(10.0)
sport_client.Init()

states_message_client = ChannelSubscriber("rt/sportmodestate", SportModeState_)
states_message_client.Init()

## 5.2 IMU

In [5]:
def get_imu_yaw():
    """获取IMU的偏航角(yaw)"""
    message = states_message_client.Read()
    if message:
        return message.imu_state.rpy[2]  # rpy[2]是yaw（弧度）
    return None

def get_imu_rpy():
    """获取IMU的偏航角(yaw)"""
    message = states_message_client.Read()
    if message:
        return message.imu_state.rpy  # rpy[2]是yaw（弧度）
    return None

def get_imu_quaternion():
    """获取IMU的偏航角(yaw)"""
    message = states_message_client.Read()
    if message:
        return message.imu_state.quaternion  # rpy[2]是yaw（弧度）
    return None

def get_current_position():
    """获取当前机器人位置"""
    message = states_message_client.Read()
    if message:
        return message.position
    return None

In [6]:
get_imu_rpy()

[-0.00160655053332448, -0.08714603632688522, -2.8044309616088867]

In [ ]:
while True:
    imu_infor = get_imu_rpy()
    # change from radius to degrees
    imu_infor_deg0 = float(imu_infor[0])/3.14*360
    imu_infor_deg1 = float(imu_infor[1])/3.14*360
    imu_infor_deg2 = float(imu_infor[2])/3.14*360
    print(imu_infor_deg0, "raw degrees", imu_infor_deg1, "pitch degrees", imu_infor_deg2, "yaw degrees")
    print("*" * 36)

## 5.3 Light Control

In [7]:
from unitree_sdk2py.core.channel import ChannelFactoryInitialize
from unitree_sdk2py.go2.vui.vui_client import VuiClient

In [ ]:
ChannelFactoryInitialize(0, "ens37")
light_client = VuiClient()
light_client.SetTimeout(3.0)
light_client.Init()

In [ ]:
print("#################GetBrightness####################")
code, level = light_client.GetBrightness()
if code != 0:
    print("get brightness error. code:", code)
else:
    print("get brightness success. level:", level)

In [ ]:
print("#################SetBrightness####################")
i=10
code = light_client.SetBrightness(i)

if code != 0:
    print("set brightness error. code:", code)
else:
    print("set brightness success. level:", i)

## 5.4 Brighting light

In [ ]:
for i in range(1, 11):
    print("#################GetBrightness####################")
    code, level = light_client.GetBrightness()

    if code != 0:
        print("get brightness error. code:", code)
    else:
        print("get brightness success. level:", level)

    time.sleep(1)

    print("#################SetBrightness####################")

    code = light_client.SetBrightness(i)

    if code != 0:
        print("set brightness error. code:", code)
    else:
        print("set brightness success. level:", i)

    time.sleep(1)

    print("#################SetBrightness 0####################")

    code  = light_client.SetBrightness(0)